<a href="https://colab.research.google.com/github/Divyesh-Kanagavel/deep_learning--keras/blob/master/Deeplearning_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Classic nlp steps:
standardization of text : depending on text remove unwanted characters,
punctuations, lowercase everything etc
Tokenization : break the text into tokens [bag of words] or a sequence of words into words / sub-words.
N-gram approach is useful when we dont need to care about sequence , but it stores some info on local ordering of words like " on the" , "to go" etc . for applications where sequence is important -> word-wise tokenizations is used.

In [ ]:
import string
class Vectorizer:
  def standardize(self,text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)
  def tokenize(self,text):
    text = self.standardize(text)
    return text.split()
  def make_vocabulary(self,dataset):
    self.vocabulary = {"" : 0, "UNK" : 1}
    for text in dataset:
      tokens = self.tokenize(text)
      for token in tokens:
        if token not in self.vocabulary:
          self.vocabulary[token] = len(self.vocabulary)
    self.inverse_vocabulary = dict((v,k) for k,v in self.vocabulary.items())

  def encode(self,text):
   tokens = self.tokenize(text)
   return [self.vocabulary.get(token,1) for token in tokens]
  def decode(self,int_sequence):
   return " ".join(self.inverse_vocabulary.get(i,"UNK") for i in int_sequence)

vectorizer = Vectorizer()
dataset = ["I write, erase, rewrite", "Erase again, and then","A poppy blossoms." ]
vectorizer.make_vocabulary(dataset)



In [ ]:
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

[2, 3, 5, 7, 1, 5, 6]
i write rewrite and UNK rewrite again


Using pure python code will not be performant and it is better to use the optimal keras library for the same . the benefit of using keras is that it can be integrated to tf.data pipeline easily. also,the standardisation and tokenization functions can be customized.

In [2]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
text_vectorization = TextVectorization(output_mode = 'int')

In [ ]:
#a sample dataset - to illustrate how vocab is created and how tokenization is done with a pre-available dataset
dataset = ["I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",]
text_vectorization.adapt(dataset) # api for vectorization object to use this dataset as base for developing vocab
text_vectorization.get_vocabulary()


['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = 'I write and then erase and then write again poppy blooms'
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  9  4  2  9  4  3 10  6  8], shape=(11,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
print(inverse_vocab)

{0: '', 1: '[UNK]', 2: 'erase', 3: 'write', 4: 'then', 5: 'rewrite', 6: 'poppy', 7: 'i', 8: 'blooms', 9: 'and', 10: 'again', 11: 'a'}


In [ ]:
decoded_sentence = ' '.join(inverse_vocab[int(i)] for i in encoded_sentence)

In [ ]:
decoded_sentence

'i write and then erase and then write again poppy blooms'

text_vectorization is only a dictionary lookup operation and is done on the and not on gpu/tpu.
hence, if this is added inside a keras model as a functional api , the rest of the model will be on gpu and will wait till the lookup is done on the cpu at each training cycle.the transfoer of data between cpu and gpu will be expensive (synchronous operation)
if this function is added in to tf.data pipeline, the text_vectorization api can be processed on batches of data asynchronously spread across multiple cores making it efficient
e.g.
int_sequence_dataset = string_dataset.map(text_vectorization, parallel_calls=4) -> spread across multiple cores.

Transformers and rnns are sequence models. there are two approaches to solve natural language problems
either discard order and treat words in a text as bag of words and process the data, or treat the words strictly based on the incoming order like steps in a timeseries and put them into rnn sort of models.
Transformers are order agnostic but in their representations, they encode some sort of word positions information.

In [3]:
!rm -r /content/aclImdb

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  27.4M      0  0:00:02  0:00:02 --:--:-- 27.4M


In [ ]:
!rm -r /content/aclImdb/train/unsup

In [ ]:
!cat /content/aclImdb/train/pos/10000_8.txt

Homelessness (or Houselessness as George Carlin stated) has been an issue for years but never a plan to help those on the street that were once considered human who did everything from going to school, work, or vote for the matter. Most people think of the homeless as just a lost cause while worrying about things such as racism, the war on Iraq, pressuring kids to succeed, technology, the elections, inflation, or worrying if they'll be next to end up on the streets.<br /><br />But what if you were given a bet to live on the streets for a month without the luxuries you once had from a home, the entertainment sets, a bathroom, pictures on the wall, a computer, and everything you once treasure to see what it's like to be homeless? That is Goddard Bolt's lesson.<br /><br />Mel Brooks (who directs) who stars as Bolt plays a rich man who has everything in the world until deciding to make a bet with a sissy rival (Jeffery Tambor) to see if he can live in the streets for thirty days without th

In [ ]:
import os, random, shutil, pathlib
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"
for category in ("neg","pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337).shuffle(files)
  num_val_samples = int(0.2*len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir/category/fname, val_dir/category/fname)


In [ ]:
from tensorflow import keras
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory(train_dir, batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(val_dir, batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('aclImdb/test', batch_size=batch_size)

there are many different ways of building the input tensors -> discard order and form a set of words in the input text. then hot encode them thus getting a single vector with zeros in other word positions and ones in words which are there in the text.
look at one word at a time - unigram or form sets of n consecutive words - n-gram thereby preserving local order information.
first basic model is unigram with a single multidimensional n-hot encoded vector passed to a neural network.
limit the vocab to 20,000 common words -> this is a heuristic, usually 20000 works well.

In [ ]:
text_vectorization = TextVectorization(max_tokens = 20000, output_mode = 'multi_hot')

In [ ]:
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)


In [ ]:
text_vectorization.get_vocabulary()

['[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'with',
 'for',
 'but',
 'movie',
 'film',
 'on',
 'not',
 'you',
 'his',
 'are',
 'have',
 'he',
 'be',
 'one',
 'its',
 'at',
 'all',
 'by',
 'an',
 'they',
 'from',
 'who',
 'so',
 'like',
 'her',
 'or',
 'just',
 'about',
 'has',
 'if',
 'out',
 'some',
 'there',
 'what',
 'good',
 'more',
 'very',
 'when',
 'even',
 'my',
 'she',
 'no',
 'up',
 'would',
 'which',
 'time',
 'only',
 'really',
 'story',
 'their',
 'were',
 'see',
 'had',
 'can',
 'me',
 'than',
 'we',
 'much',
 'well',
 'been',
 'get',
 'also',
 'into',
 'will',
 'other',
 'do',
 'great',
 'bad',
 'people',
 'because',
 'first',
 'most',
 'how',
 'him',
 'dont',
 'made',
 'then',
 'movies',
 'could',
 'films',
 'make',
 'way',
 'any',
 'after',
 'too',
 'them',
 'characters',
 'think',
 'watch',
 'many',
 'two',
 'being',
 'seen',
 'little',
 'character',
 'never',
 'best',
 'plot',
 'where',
 'acting',


In [ ]:
binary_1gram_train_ds = train_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4 )
binary_1gram_val_ds = val_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4 )
binary_1gram_test_ds = test_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4 )

In [ ]:
for inputs,targets in binary_1gram_train_ds:
  print(inputs.shape)
  print(targets.shape)
  print(inputs[0].dtype)
  print(targets[0].shape)
  break

(32, 20000)
(32,)
<dtype: 'float32'>
()


Reusable model building utility

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def get_model(max_tokens = 20000,hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))
  x = layers.Dense(hidden_dim, activation="relu") (inputs)
  x = layers.Dropout(0.5) (x)
  outputs = layers.Dense(1, activation="sigmoid") (x)
  model = keras.Model(inputs, outputs)
  model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
model = get_model()

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_1gram.h5",
                                    save_best_only=True)
]

In [ ]:
model.fit(binary_1gram_train_ds.cache(),validation_data = binary_1gram_val_ds.cache(), epochs = 10, callbacks = callbacks)

Epoch 1/10
625/625 [==============================] - 9s 9ms/step - loss: 0.4106 - accuracy: 0.8283 - val_loss: 0.2940 - val_accuracy: 0.8824
Epoch 2/10
 46/625 [=>............................] - ETA: 2s - loss: 0.2947 - accuracy: 0.8886

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 3s 4ms/step - loss: 0.2773 - accuracy: 0.8975 - val_loss: 0.2891 - val_accuracy: 0.8852
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2393 - accuracy: 0.9166 - val_loss: 0.3003 - val_accuracy: 0.8870
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2231 - accuracy: 0.9255 - val_loss: 0.3164 - val_accuracy: 0.8866
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2103 - accuracy: 0.9294 - val_loss: 0.3308 - val_accuracy: 0.8848
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2057 - accuracy: 0.9303 - val_loss: 0.3493 - val_accuracy: 0.8862
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2039 - accuracy: 0.9337 - val_loss: 0.3538 - val_accuracy: 0.8862
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2027 - accuracy: 0.9359 - val_loss: 0.3660 - val_accuracy: 0.8838
Epoch 9/10
625/62

binary_1gram_train_ds is a map from train_ds.so everytime an epoch is called, the strings from train_ds are taken in batches, text vectorization is done on it [map function], then fed to the weights which are loaded in the gpu. this preprocessing if done in every epoch is redundant and happens on the cpu making the gpu wait.instead we call binary_1gram_train_ds.cache() to store the preprocessed data in cache memory during first epoch [works for smaller sized data]

an accuracy of 88.7 percent is achieved with this simple model itself beating the baseline accuracy of 50 percentage , which is cool.the objective is to increase the accuracy to as much as possible.

In [ ]:
model = keras.models.load_model("binary_1gram.h5")
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

782/782 [==============================] - 5s 6ms/step - loss: 0.2876 - accuracy: 0.8864
Test acc: 0.886


In [ ]:
# train_ds object is an iterator object of fixed bytes which loops through data and returns batch size data to the caller.
# so the entire data is not stored as arrays [creating copies], they are processed on the fly in the cpus during training / inference
#hence the need to cache it during training
#import sys
#sys.getsizeof(binary_1gram_train_ds)


Bigram encoding : even simple pair of words can carry context like 'stand up' , sit down etc which often come in pairs.

In [ ]:
text_vectorization = TextVectorization(ngrams=2,max_tokens= 20000,output_mode="multi_hot")

In [ ]:
text_vectorization.adapt(text_only_train_ds)

In [ ]:
text_vectorization.get_vocabulary()[:20]

['[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'with',
 'for',
 'but',
 'movie',
 'of the']

In [ ]:
binary_2gram_train_ds = train_ds.map(
lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
lambda x, y: (text_vectorization(x), y),
    num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

In [ ]:
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("binary_2gram.h5",
                                    save_best_only=True)
]

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
model.fit(binary_2gram_train_ds.cache(), validation_data=binary_2gram_val_ds.cache(), epochs=10,
callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 9s 9ms/step - loss: 0.3854 - accuracy: 0.8397 - val_loss: 0.2827 - val_accuracy: 0.8902


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2428 - accuracy: 0.9130 - val_loss: 0.2825 - val_accuracy: 0.8948
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2028 - accuracy: 0.9323 - val_loss: 0.3062 - val_accuracy: 0.8916
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1813 - accuracy: 0.9431 - val_loss: 0.3161 - val_accuracy: 0.8960
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1742 - accuracy: 0.9484 - val_loss: 0.3513 - val_accuracy: 0.8890
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1617 - accuracy: 0.9509 - val_loss: 0.3508 - val_accuracy: 0.8944
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1629 - accuracy: 0.9521 - val_loss: 0.3577 - val_accuracy: 0.8934
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.1538 - accuracy: 0.9560 - val_loss: 0.3656 - val_accuracy: 0.8962
Epoch 

In [ ]:
model = keras.models.load_model("binary_2gram.h5")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 6s 7ms/step - loss: 0.2792 - accuracy: 0.8952
Test acc: 0.895


an improvement of 1 percent accuracy with bigram model!

let us try bigram encoding with tf-idf : term frequency - inverse document frequency
the sparsity property is embedded in our bigram multi-hot encoded representation
that is only those terms which appear in the document frequently are given 1 and others are zeros -> this is a useful property which reduces computational load and also prevents overfitting.
one way of introducing normalization is subtracting the mean of the term score appearing across documents and dividing the variance -> feature normalization, but this would disturb normalization.hence we need to divide by a quantity alone
here is where tf-idf comes in., there are some terms which appear many times in the document like a, the, as, which don't contribute much towards sentiment classification.
hence , we can find document frequency -> number of times the word appears across the documents and divide by the term frequency, so that unique words which matter to this document are picked up.

def tf_idf(term, document, dataset):
    term_freq = document.count(term)
    doc_freq = math.log(sum(doc.count(term) for doc in dataset)+1)
    return term_freq/doc_freq
    

the tf-idf is build as a functionality in keras text_vectorization module.

In [ ]:
text_vectorization = TextVectorization(ngrams=2, max_tokens = 20000, output_mode = "tf-idf")

In [ ]:
text_vectorization.adapt(text_only_train_ds)

In [ ]:
# prompt: create tf_idf ds for train, val and test as a map to train_ds, val_ds, test_ds with cpu threading num_parallel calls=4

tf_idf_2gram_train_ds = train_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4)
tf_idf_2gram_val_ds = val_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4)
tf_idf_2gram_test_ds = test_ds.map(lambda x,y : (text_vectorization(x),y), num_parallel_calls=4)


In [ ]:
model = get_model()

In [ ]:
# prompt: get summary of the model

model.summary()


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# prompt: create a callback to store the model as 'tf_idf_2gram.h5' with save_best_only as True

callbacks = [
    keras.callbacks.ModelCheckpoint("tf_idf_2gram.h5",
                                    save_best_only=True)
]


In [ ]:
model.fit(tf_idf_2gram_train_ds.cache(),
validation_data=tf_idf_2gram_val_ds.cache(), epochs=10,
callbacks=callbacks)

Epoch 1/10
625/625 [==============================] - 7s 10ms/step - loss: 0.4889 - accuracy: 0.7930 - val_loss: 0.3317 - val_accuracy: 0.8486


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3196 - accuracy: 0.8737 - val_loss: 0.3043 - val_accuracy: 0.8774
Epoch 3/10
625/625 [==============================] - 3s 6ms/step - loss: 0.2730 - accuracy: 0.8910 - val_loss: 0.3330 - val_accuracy: 0.8884
Epoch 4/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2491 - accuracy: 0.8972 - val_loss: 0.3441 - val_accuracy: 0.8736
Epoch 5/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2432 - accuracy: 0.9018 - val_loss: 0.3419 - val_accuracy: 0.8754
Epoch 6/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2299 - accuracy: 0.9064 - val_loss: 0.3516 - val_accuracy: 0.8764
Epoch 7/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2194 - accuracy: 0.9075 - val_loss: 0.3787 - val_accuracy: 0.8698
Epoch 8/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2111 - accuracy: 0.9090 - val_loss: 0.3910 - val_accuracy: 0.8666
Epoch 

In [ ]:
# prompt: based on model.fit() results above, put your inference and analysis as a markdown text

The model.fit() results show that the model trained on the tf-idf_2gram_train_ds dataset achieved a test accuracy of 89.9%. This is an improvement over the test accuracy of 88.7% achieved by the model trained on the binary_1gram_train_ds dataset and the test accuracy of 89.2% achieved by the model trained on the binary_2gram_train_ds dataset.

This suggests that using tf-idf weighting can be beneficial for sentiment classification tasks, as it can help to identify and emphasize the most important words and phrases in the text.

Further improvements to the model could be explored by experimenting with different hyperparameters, such as the number of epochs, the learning rate, and the number of hidden units in the neural network. Additionally, other types of word embeddings, such as word2vec or GloVe, could be used to represent the words in the text.

In [ ]:
model = keras.models.load_model("tf_idf_2gram.h5")
print(f"Test acc: {model.evaluate(tf_idf_2gram_test_ds)[1]:.3f}")

782/782 [==============================] - 4s 5ms/step - loss: 0.2962 - accuracy: 0.8803
Test acc: 0.880


it is important to be careful with gen AI generated code and commentaries -> though it is very useful for asistance in finding APIs, hyperparameters, if it is not sure of something, it confidently puts a wrong number, for example it confidently claims that an accuracy of 89.9 percent was obtained with tf-idf ,whereas 88.03 is the actual test accuracy and for this tf-idf does not produce a better accuracy but usualy it produces better accuracy for say large datasets.


Sequential models : Treat the text as a sequence of integer indices which will be mapped to a corresponding vector embedding.

In [44]:
from tensorflow.keras import layers
max_length = 600 # truncate length of reviews to 600
max_tokens = 20000 # max tokens in vocab
text_vectorization = layers.TextVectorization(max_tokens = max_tokens, output_mode = "int", output_sequence_length=max_length)
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)


In [ ]:
text_vectorization.get_vocabulary() [:20]

['',
 '[UNK]',
 'the',
 'and',
 'a',
 'of',
 'to',
 'is',
 'in',
 'it',
 'i',
 'this',
 'that',
 'br',
 'was',
 'as',
 'for',
 'with',
 'movie',
 'but']

In [45]:
int_train_ds = train_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_val_ds = val_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)
int_test_ds = test_ds.map(
lambda x, y: (text_vectorization(x), y), num_parallel_calls=4)

one-hot encode the integer sequences :
embedding size becomes (600,20000)

In [ ]:
for i,j in int_train_ds:
  print(j.shape)
  break

(32,)


In [ ]:
import tensorflow as tf
inputs = keras.Input(shape=(None,),dtype='int64')
embedded = tf.one_hot(inputs,depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32)) (embedded)
x = layers.Dropout(0.5) (x)
output = layers.Dense(1, activation='sigmoid') (x)
model = keras.Model(inputs = inputs, outputs= output)


In [ ]:
# prompt: compile the model with rmsprop optimiser, metrics as accuracy and loss function as binary cross entropy

model.compile(optimizer='rmsprop', metrics=['accuracy'], loss='binary_crossentropy')


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot_2 (TFOpLambda)   (None, None, 20000)       0         
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                5128448   
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5128513 (19.56 MB)
Trainable params: 5128513 (19.56 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.h5",
                                    save_best_only=True)
]

the training takes a lot of time because each sequence is 600 words and embedding size is 200000. whereas in bigram model, it was a bunch of bigrams stored in dictionary and used in a de

In [ ]:
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)


Epoch 1/10
625/625 [==============================] - 164s 252ms/step - loss: 0.5567 - accuracy: 0.7245 - val_loss: 0.4347 - val_accuracy: 0.8054
Epoch 2/10


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 165s 264ms/step - loss: 0.3635 - accuracy: 0.8666 - val_loss: 0.3449 - val_accuracy: 0.8602
Epoch 3/10
625/625 [==============================] - 164s 262ms/step - loss: 0.2936 - accuracy: 0.8959 - val_loss: 0.3348 - val_accuracy: 0.8640
Epoch 4/10
625/625 [==============================] - 166s 266ms/step - loss: 0.2460 - accuracy: 0.9147 - val_loss: 0.3061 - val_accuracy: 0.8910
Epoch 5/10
625/625 [==============================] - 165s 264ms/step - loss: 0.2166 - accuracy: 0.9261 - val_loss: 0.3742 - val_accuracy: 0.8832
Epoch 6/10
625/625 [==============================] - 166s 266ms/step - loss: 0.2004 - accuracy: 0.9354 - val_loss: 0.2869 - val_accuracy: 0.8856
Epoch 7/10
625/625 [==============================] - 165s 263ms/step - loss: 0.1694 - accuracy: 0.9443 - val_loss: 0.3334 - val_accuracy: 0.8752
Epoch 8/10
625/625 [==============================] - 166s 266ms/step - loss: 0.1501 - accuracy: 0.9530 - val_loss: 0.3449 - val_accura

87.3 percent accuracy is observed, which is worse than the fast bigram model -> clearly the model is struggling to process (600, 20000) for each review

Understand word embeddings : One-hot encoding is a sort of feature engineering -> we are injecting an assumption about the word structure that they are independent of one another , one hot encoded vectors are all orthogonal to one another. Reality is very different -> the words are all related to one another either semantically similar or dissimilar or sometimes antagonastic. word embeddings try to capture semantic similarity of words in terms of geometric patterns.
one-hot encoded vector : sparse -> boolean -> high dimensional
embedding vector : dense -> float -> low dimensional
word-embedding are also learned instead of hardcoded.
word embeddings are as real asit gets if trained properly -> with female vector added to king, we go to embedding of queen etc

Word embeddings could be task specific or pre-trained from another task.

In [ ]:
embedding_layer = layers.Embedding(input_dim = max_tokens, output_dim = 256)

Embedding layer is a dictionary lookup which maps integer indices to vector

In [ ]:
inputs = keras.Input(shape=(None,), dtype='int64')
embedded = layers.Embedding(input_dim = max_tokens, output_dim = 256) (inputs)
x = layers.Bidirectional(layers.LSTM(32)) (embedded)
x= layers.Dropout(0.5) (x)
outputs = layers.Dense(1, activation="sigmoid") (x)
model = keras.Model(inputs = inputs, outputs = outputs)



In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
_____________________

In [ ]:
model.compile(optimizer='rmsprop', metrics = ['accuracy'], loss = 'binary_crossentropy')

In [ ]:
# prompt: creata a kerbs callback object with model checkpoint saved to 'bidirectional_embedding.h5' file with best model saved

callbacks = [
    keras.callbacks.ModelCheckpoint("bidirectional_embedding.h5",
                                    save_best_only=True)
]


In [ ]:
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)

Epoch 1/10
625/625 [==============================] - 64s 91ms/step - loss: 0.5340 - accuracy: 0.7369 - val_loss: 0.4561 - val_accuracy: 0.8244


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 2/10
625/625 [==============================] - 41s 65ms/step - loss: 0.3576 - accuracy: 0.8659 - val_loss: 0.4805 - val_accuracy: 0.7928
Epoch 3/10
625/625 [==============================] - 36s 58ms/step - loss: 0.2846 - accuracy: 0.8959 - val_loss: 0.3269 - val_accuracy: 0.8838
Epoch 4/10
625/625 [==============================] - 31s 49ms/step - loss: 0.2417 - accuracy: 0.9173 - val_loss: 0.3490 - val_accuracy: 0.8728
Epoch 5/10
625/625 [==============================] - 31s 49ms/step - loss: 0.2064 - accuracy: 0.9298 - val_loss: 0.3447 - val_accuracy: 0.8702
Epoch 6/10
625/625 [==============================] - 32s 52ms/step - loss: 0.1767 - accuracy: 0.9427 - val_loss: 0.3900 - val_accuracy: 0.8674
Epoch 7/10
625/625 [==============================] - 29s 47ms/step - loss: 0.1536 - accuracy: 0.9501 - val_loss: 0.4924 - val_accuracy: 0.8556
Epoch 8/10
625/625 [==============================] - 29s 46ms/step - loss: 0.1274 - accuracy: 0.9589 - val_loss: 0.4343 - val_accuracy:

In [ ]:
model = keras.models.load_model("bidirectional_embedding.h5")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

782/782 [==============================] - 17s 21ms/step - loss: 0.3403 - accuracy: 0.8754
Test acc: 0.875


the accuracy is still not that great it is just above 87.5 percent but the model with word embedding train much faster. still not better tha bigram model.
one reason could be the truncation of review to 600 words. maybe some information is being lost. also , if the number of words is less than 600, they are padded with 0, so an bidirectional lstm which processes data in both natural order and reverse order, we see that some tokens are zeros and the original meaning which was learnt from the initial words are getting lost with the meaningless inputs. we need to mask the zeros to feed the rnn with meaningful words only.

in embedding api, mask_Zero is available, which masks the places which have zero values, skipping those for RNN computations
Keras will pass the masks as metadata to every layer which process the data.in case sequnce of data is passed to the loss function during training, the masked portions will be skiiped to compute loss

In [ ]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
input_dim=max_tokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
loss="binary_crossentropy",
metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirecti  (None, 64)                73984     
 onal)                                                           
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5194049 (19.81 MB)
Trainable params: 5194049 (19.81 MB)
Non-trainable params: 0 (0.00 Byte)
___________________

In [ ]:
# prompt: creata a cerasa callback function to save model checkpoint with name "lstm_embedding_masked.h5" saving the best model.

callbacks = [
    keras.callbacks.ModelCheckpoint("lstm_embedding_masked.h5",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)


Epoch 1/10
625/625 [==============================] - 67s 94ms/step - loss: 0.4431 - accuracy: 0.7904 - val_loss: 0.3250 - val_accuracy: 0.8660
Epoch 2/10
625/625 [==============================] - 42s 68ms/step - loss: 0.2816 - accuracy: 0.8899 - val_loss: 0.2935 - val_accuracy: 0.8726
Epoch 3/10
625/625 [==============================] - 40s 65ms/step - loss: 0.2210 - accuracy: 0.9165 - val_loss: 0.2970 - val_accuracy: 0.8812
Epoch 4/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1706 - accuracy: 0.9386 - val_loss: 0.3487 - val_accuracy: 0.8822
Epoch 5/10
625/625 [==============================] - 34s 55ms/step - loss: 0.1378 - accuracy: 0.9500 - val_loss: 0.3346 - val_accuracy: 0.8864
Epoch 6/10
625/625 [==============================] - 34s 54ms/step - loss: 0.1043 - accuracy: 0.9647 - val_loss: 0.4329 - val_accuracy: 0.8782
Epoch 7/10
625/625 [==============================] - 30s 49ms/step - loss: 0.0808 - accuracy: 0.9732 - val_loss: 0.3842 - val_accuracy:

just like how for vision applications, pretrained convnets were used with finetuning applied on problem specific dataset, pre-trained embeddings trained on vast corpus of data can be used as embeddings whcih capture generic language patterns and semantic features. Such pre-trained embeddings may have been done using word-statistic analysis - like words occur togetehr across documetns etc or a neural network dedicated to it.
Word2Vec is one of the famous word-embedding algorithm. Another famous word embedding scheme is Global vectors for word representation - Glove


Glove - 100 dimensional embedding for 400000 words

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2024-04-20 03:12:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-04-20 03:12:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-04-20 03:12:50--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
!unzip -q glove.6B.zip

Parsing the glove text file into a dictionary

In [ ]:
import numpy as np
path_to_glove_file = "glove.6B.100d.txt"

In [ ]:
embeddings_index = {}
with open(path_to_glove_file) as f:
  for line in f:
    word, coefs = line.split(maxsplit=1)
    coefs = np.fromstring(coefs, dtype='float', sep=" ")
    embeddings_index[word] = coefs
print(f"found {len(embeddings_index)} word vectors")


found 400000 word vectors


In [ ]:
embedding_dim = 100
vocabulary = text_vectorization.get_vocabulary()
print(len(vocabulary))


20000


In [ ]:
word_index = dict(zip(vocabulary, range(len(vocabulary))))
len(word_index
    )

20000

In [ ]:
vocabulary[:5]

['', '[UNK]', 'the', 'and', 'a']

In [ ]:
embedding_matrix = np.zeros((max_tokens,embedding_dim))

In [ ]:
for word, i in word_index.items():
  if i < max_tokens:
    embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
embedding_layer = layers.Embedding(max_tokens,embedding_dim,embeddings_initializer=keras.initializers.constant(embedding_matrix), trainable=False, mask_zero=True)

this pretrained embedding shoudl not be distrubed during training, so we use trainable = false for embedding layer and load this constant embedding matrix as initializer

In [ ]:
# prompt: write a keras model which symbolic input data , has a LSTM with 32 hidden units in bidirectional mode, has a dropout of 0.5 as next layer and the final layer as dense layer with 1 neutron and sigmoid activation

inputs = keras.Input(shape=(None,), dtype='int64')
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32)) (embedded)
x = layers.Dropout(0.5) (x)
output = layers.Dense(1, activation='sigmoid') (x)
model = keras.Model(inputs = inputs, outputs= output)
model.compile(optimizer='rmsprop', metrics=['accuracy'], loss='binary_crossentropy')
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         2000000   
                                                                 
 bidirectional (Bidirection  (None, 64)                34048     
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 2034113 (7.76 MB)
Trainable params: 34113 (133.25 KB)
Non-trainable params: 2000000 (7.63 MB)
___________________

In [ ]:
# prompt: creat a keras callback to store model checkpoint and save the best model to pretrained_embed_gru.h5

callbacks = [
    keras.callbacks.ModelCheckpoint("pretrained_embed_gru.h5",
                                    save_best_only=True)
]


In [ ]:
model.fit(int_train_ds, validation_data = int_val_ds, epochs = 10, callbacks = callbacks)

Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.5783 - accuracy: 0.6910

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


625/625 [==============================] - 56s 68ms/step - loss: 0.5782 - accuracy: 0.6909 - val_loss: 0.4778 - val_accuracy: 0.7676
Epoch 2/10
625/625 [==============================] - 39s 63ms/step - loss: 0.4480 - accuracy: 0.7955 - val_loss: 0.4031 - val_accuracy: 0.8194
Epoch 3/10
625/625 [==============================] - 56s 89ms/step - loss: 0.3995 - accuracy: 0.8269 - val_loss: 0.3995 - val_accuracy: 0.8228
Epoch 4/10
625/625 [==============================] - 39s 62ms/step - loss: 0.3675 - accuracy: 0.8435 - val_loss: 0.3843 - val_accuracy: 0.8346
Epoch 5/10
625/625 [==============================] - 37s 60ms/step - loss: 0.3466 - accuracy: 0.8544 - val_loss: 0.3429 - val_accuracy: 0.8462
Epoch 6/10
625/625 [==============================] - 47s 76ms/step - loss: 0.3261 - accuracy: 0.8632 - val_loss: 0.3530 - val_accuracy: 0.8450
Epoch 7/10
625/625 [==============================] - 41s 65ms/step - loss: 0.3104 - accuracy: 0.8719 - val_loss: 0.3246 - val_accuracy: 0.8650
Epo

there is no difference with using pre-trained embeddings for this use-case. usually for smalelr datasets, pre-trained embedding give a boost in accuracy,but in this case the dataset itself contains enough information

The transformer architecture

In [ ]:
def self_attention(input_sequence):
  output = np.zeros(shape=input_sequence.shape)
  for i, pivot_vector in enumerate(input_sequence):
    scores = np.zeros(shape=(len(input_sequence),))
    for j, vector in enumerate(input_sequence):
      scores[j] = np.dot(pivot_vector, vector.T)
    scores /= np.sqrt(input_sequence.shape[1])
    scores = softmax(scores)
    new_pivot_vector = np.zeros(shape=pivot_vector.shape)
    for j, vector in enumerate(input_sequence):
      new_pivot_vector += vector * scores[j]
    output[i] = new_pivot_vector
  return output



in practice, however a vectorized implementation is used and keras has a layer -> multihead attention to do it for us.
Transformer architecture made use of every method used to train deep neural networks
-> representation of output latent space refactored into independent sub spaces [multihead attention]
-> skip connections to prevent vanishing/exploding gradients [residual networks]
-> layernorm to prevent internal covariate shift

Transformer.encoder - process the source sequence
Transformer decoder - use the processed sequence to generate translated version
in the original paper used for machine translation application

Transformer encoder can be used for text classification since it can ingest a sequence , process it and use it for other tasks well.

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [56]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
       super().__init__(**kwargs)
       self.embed_dim = embed_dim
       self.dense_dim = dense_dim
       self.num_heads = num_heads
       self.attention = layers.MultiHeadAttention(
       num_heads=num_heads, key_dim=embed_dim)
       self.dense_proj = keras.Sequential(
    [layers.Dense(dense_dim, activation="relu"),
     layers.Dense(embed_dim),])
       self.layernorm_1 = layers.LayerNormalization()
       self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
      if mask is not None:


        mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
    def get_config(self):
        config = super().get_config()
        config.update({

           "embed_dim": self.embed_dim,
           "num_heads": self.num_heads,
           "dense_dim": self.dense_dim,
             })






storing the config params of a layer in a dict will be useful during saving/loading a model.
Example:
layer = PositionalEmbedding(sequence_length, input_dim, output_dim)
config = layer.get_config()
new_layer = PositionalEmbedding.from_config(config)

layer's configs are stored as dictionary and can be used by another layer with the config params loaded.

In [1]:
def layer_normalization(batch_of_sequences):
    mean = np.mean(batch_of_sequences, keepdims=True, axis=-1)
    variance = np.var(batch_of_sequences, keepdims=True, axis=-1)
    return (batch_of_sequences - mean) / variance

layer normalization is done along the last axis . batch_of_sequences -> batch_size, input_sequence, embedding_dim. so, the mean and var are computed for embedding dim vector for each input_sequence word.

Using the transformer encoder for text classification


In [ ]:
vocab_size = 20000
embed_dim = 256
num_heads = 2
dense_dim = 32

inputs = keras.Input(shape=(None,), dtype="int64")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(inputs)
mask = layers.Embedding(vocab_size, embed_dim, mask_zero=True).compute_mask(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x,mask)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)



In [48]:
# prompt: compile the model with rmsprop, loss as binarycross entropy and metrics as accuracy

model.compile(optimizer='rmsprop', loss='binary_crossentropy',
metrics=['accuracy'])


In [49]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_7 (Embedding)     (None, None, 256)         5120000   
                                                                 
 transformer_encoder_7 (Tra  (None, None, 256)         543776    
 nsformerEncoder)                                                
                                                                 
 global_max_pooling1d_5 (Gl  (None, 256)               0         
 obalMaxPooling1D)                                               
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_18 (Dense)            (None, 1)                 257 

In [50]:
callbacks = [keras.callbacks.ModelCheckpoint("transformer_encoder.h5", save_best_only=True)]